In [ ]:
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import os,imageio,glob
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from scipy.interpolate import interp1d
import pandas as pd
from PIL import Image
import time

In [ ]:
#User arguments
IMAGE_DIR = "/home/mars/Projects/scratch_assays_scott/" #Where the raw image files are located
DATA_DIR = "../final_trajectories/" #Where the dataframes are located
FIGURE_DIR = "./Figures/" #Where to save the figures
if not os.path.isdir(FIGURE_DIR):
    os.makedirs(FIGURE_DIR)
NUM_STEPS = 10 #How many steps to keep
Zoom = True #Whether or not to save a zoomed-in version
IDs = False #Whether or not to plot particle labels
INST_ARROWS = False #Whether or not to plot instantaneous velocity arrows
DISP_ARROWS = False #Whether or not to plot displacement arrows
CIRC_SIZE = 8 #Size of current location marker

#Get matplotlib colormap for angles
my_cmap = cm.get_cmap('twilight')
my_norm = Normalize(vmin=-np.pi, vmax=np.pi)

color_ticks = np.arange(np.pi,-np.pi-np.pi/4,-np.pi/4)
color_tick_labels = ["$-\pi$","$-3\pi/4$","$-\pi/2$","$-\pi/4$","0","$\pi/4$","$\pi/2$","$3\pi/4$","$\pi$"]

#loop through stages
for stage_num in np.arange(1,42+1):
    beg = time.time()
    entire_frames = []
    if Zoom:
        zoomed_frames = []
    t = pd.read_pickle(os.path.join(DATA_DIR,"stage_"+str(stage_num)+".pkl"))
    drift = pd.read_pickle(os.path.join(DATA_DIR,"pixel_drift","drift_pixels_"+str(stage_num)+".pkl"))
    frame_shift = 0#correct for missing frames in each stage

    #loop through frames
    for i in np.arange(1,129):
        if i >= 128-frame_shift:
            continue

        particles = t[t['frame']==i]
        fig = plt.figure(figsize=(13,8))
        ax = fig.add_subplot(111)

        for j in particles.particle.unique():

            #one particle over time
            particle = t[t['particle']==j]

            time_vec = particle['frame']
            x = particle['x'][time_vec<=i]
            y = particle['y'][time_vec<=i]
            vx = particle['vx'][time_vec<=i]
            vy = particle['vy'][time_vec<=i]

            #Angle from initial to final
            displacement = (x.iloc[-1]-x.iloc[0],y.iloc[-1]-y.iloc[0])
            smooth_angle = np.arctan2(displacement[1],displacement[0])
            displacement = displacement if np.linalg.norm(displacement) == 0 else displacement/np.linalg.norm(displacement)
            #Angle vector from pointwise velocities
            pointwise_angle = particle['angle'][time_vec<=i].iloc[-1]

            #don't show entire trajectory, just previous NUM_STEPS locations
            if len(x) > NUM_STEPS:
                x = x[-NUM_STEPS:]
                y = y[-NUM_STEPS:]

            ax.plot(x,y,'-',color='r')#my_cmap(my_norm(angle)))
            ax.plot(x.iloc[-1],y.iloc[-1],'o',ms=CIRC_SIZE,mfc='none',mec='r')#mec = np.array(my_cmap(my_norm(angle))))
            if INST_ARROWS == True:
                ax.arrow(x.iloc[-1],y.iloc[-1],vx.iloc[-1],vy.iloc[-1],head_width=12,
                     color=np.array(my_cmap(my_norm(pointwise_angle))))
                
            if DISP_ARROWS == True:
                ax.arrow(x.iloc[-1],y.iloc[-1],displacement[0],displacement[1],head_width=15,
                     edgecolor=np.array(my_cmap(my_norm(smooth_angle))),facecolor='none')

            if IDs == True:
                ax.annotate(str(particle['particle'].iloc[0]),(x.iloc[-1],y.iloc[-1]),c="w")

        #create frame image
        ax.set_title(str(i))

        if (INST_ARROWS == True) or (DISP_ARROWS == True):
            cb = plt.colorbar(cm.ScalarMappable(norm=my_norm,cmap=my_cmap),ax=ax,ticks=color_ticks)
            tick_texts = cb.ax.set_yticklabels(color_tick_labels)

        for attempt in range(10):#If the current frame is missing, skip it and put the above plot on the next one
            try:
                im_path = os.path.join(IMAGE_DIR,"scratch3t3_1nMPDGF1_w2Cy5_s"+str(stage_num)+"_t"+str(i+frame_shift)+".TIF")
                im = Image.open(im_path)
            except FileNotFoundError:
                print(im_path,"does not exist.")
                frame_shift += 1
            else:
                break
        new_im = Image.new(im.mode,im.size)
        #Shift base image according to drift
        if not i == 128:
            new_im.paste(im,(-drift.iloc[i-1].x,-drift.iloc[i-1].y))
        else:
            new_im.paste(im,(-drift.iloc[i-2].x,-drift.iloc[i-2].y))
        ax.imshow(new_im,cmap="gist_gray",vmin=510,vmax=1000,interpolation='none',
                  extent=[0,1344/(1.157*1000),1024/(1.157*1000),0])

        ax.set_xlim(0,1344/(1.157*1000))
        ax.set_ylim(1024/(1.157*1000),0)
        fig_path = FIGURE_DIR+"frame_"+str(i)+".png"
        plt.tight_layout()
        plt.savefig(fig_path)
        entire_frames.append(imageio.imread(fig_path))
        
        if Zoom:
            ax.set_xlim(1000/(1.157*1000),1200/(1.157*1000))
            ax.set_ylim(1000/(1.157*1000),800/(1.157*1000))
            fig_path = FIGURE_DIR+"zoomed_frame_"+str(i)+".png"
            plt.tight_layout()
            plt.savefig(fig_path)
            zoomed_frames.append(imageio.imread(fig_path))

        plt.close()

    gif_path = os.path.join(FIGURE_DIR,"stage_"+str(stage_num)+"_traj.gif")
    imageio.mimsave(gif_path,entire_frames,fps=5)
    if Zoom:
        gif_path = os.path.join(FIGURE_DIR,"stage_"+str(stage_num)+"_traj_ZOOM.gif")
        imageio.mimsave(gif_path,zoomed_frames,fps=5)

    #Delete all figures, important for assuring previous stage frames don't get tacked on!
    files = glob.glob(FIGURE_DIR+"/*.png")
    for f in files:
        os.remove(f)
    print("{0:.1f} minutes for stage {1}".format((time.time()-beg)/60,stage_num))